In [2]:
import pandas as pd

# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Calculate the Points_Percent_Diff column and round to five decimal places
data['Points_Percent_Diff'] = round(data['Points_Diff'] / data['Points_Running_Avg'], 5)

# Calculate the Rebounds_Percent_Diff column and round to five decimal places
data['Rebounds_Percent_Diff'] = round(data['Rebounds_Diff'] / data['Rebounds_Running_Avg'], 5)

# Calculate the Assists_Percent_Diff column and round to five decimal places
data['Assists_Percent_Diff'] = round(data['Assists_Diff'] / data['Assists_Running_Avg'], 5)

# Write the updated dataframe to percent_diff.csv
data.to_csv('percent_diff.csv', index=False)


In [13]:
# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Group the data by Player column
grouped_data = data.groupby('Player')

# Calculate the standard deviation of Points_Running_Avg, Rebounds_Running_Avg, and Assists_Running_Avg
std_points = round(grouped_data['PTS'].std(), 5)
std_rebounds = round(grouped_data['TRB'].std(), 5)
std_assists = round(grouped_data['AST'].std(), 5)

# Create new columns for the standard deviation of each
data['Points_Running_Avg_Std'] = data['Player'].map(std_points)
data['Rebounds_Running_Avg_Std'] = data['Player'].map(std_rebounds)
data['Assists_Running_Avg_Std'] = data['Player'].map(std_assists)

# Write the updated dataframe to percent_diff.csv
data.to_csv('stdev.csv', index=False)



In [14]:

# Read the stdev.csv file
data = pd.read_csv('stdev.csv')

# Calculate the z-score for PTS
data['PTS_z'] = round((data['PTS'] - data['Points_Running_Avg']) / data['Points_Running_Avg_Std'], 5)

# Calculate the z-score for TRB
data['TRB_z'] = round((data['TRB'] - data['Rebounds_Running_Avg']) / data['Rebounds_Running_Avg_Std'], 5)

# Calculate the z-score for AST
data['AST_z'] = round((data['AST'] - data['Assists_Running_Avg']) / data['Assists_Running_Avg_Std'], 5)

# Write the updated dataframe to z_score.csv
data.to_csv('z_score.csv', index=False)


In [15]:
import numpy as np

In [16]:
# Read the z_score.csv file
data = pd.read_csv('z_score.csv')

# Add columns for bet results
data['PTS_Bet_Res'] = np.where(data['PTS'] > data['player_points_line'], 'Over', 'Under')
data['TRB_Bet_Res'] = np.where(data['TRB'] > data['player_rebounds_line'], 'Over', 'Under')
data['AST_Bet_Res'] = np.where(data['AST'] > data['player_assists_line'], 'Over', 'Under')

# Write the updated dataframe to bet_result.csv
data.to_csv('bet_result.csv', index=False)

